In [2]:
from scipy import stats

from blm_activity_db import BlmActivityDb
from tweet_mgr import Stance

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
# get sentiment for each account by stance in a period
query = \
"""
SELECT 
    Stance, AA.Sentiment
FROM
    AccountActivity AA JOIN Community C 
ON
    AA.CommunityId = C.CommunityId
WHERE
    AA.PeriodId = ?
AND
    C.PeriodId = ?
"""
def get_account_sentiment_by_stance(period: int):
    """Returns Tuple[List[float], List[float]] for (activists, counter-protesters)"""
    blm_scores, c_scores = [], []
    db = BlmActivityDb()
    with db.conn as conn:
        cur = conn.cursor()
        cur.execute(query, (period, period))
        rows = cur.fetchall()
    for row in rows:
        if row[0] == Stance.Protest:
            blm_scores.append(row[1])
        elif row[0] == Stance.CounterProtest:
            c_scores.append(row[1])
    return blm_scores, c_scores

scores_tuples = []
for i in range(1, 7):
    scores_tuples.append(get_account_sentiment_by_stance(i))


In [17]:
for tup in scores_tuples:
    t, p = stats.ttest_ind(tup[0], tup[1], equal_var=False)
    print(t, p)

21.3270709416587 2.2915099295694744e-76
41.042864280428994 0.0
71.54591779565308 0.0
-20.142805229351573 4.374238521134395e-24
136.81543812827917 0.0
94.31817708468536 0.0
